In [1]:
import json
from functools import cache, cached_property
from pathlib import Path
import tempfile

import cadquery as cq
from jupyter_cadquery import show
import numpy as np
import splinecloud_scipy as sc

from cquav.turbofan import AirfoilSection, Turbofan, get_refined_airfoils_collection

Overwriting auto display for cadquery Workplane and Shape


## Get airfoils collection

In [2]:
airfoils = get_refined_airfoils_collection()

## Specify sections

In [3]:
sections = [
    AirfoilSection(airfoils["ONERA NACA CAMBRE AIRFOIL (ncambre-il)"], 0.6, 0, -20),
    AirfoilSection(airfoils["NACA 0012-64 a=0.8 c(li)=0.2 (naca001264a08cli02-il)"], 1.0, 2, -12),
    AirfoilSection(airfoils["NACA 6-H-15 AIRFOIL (n6h15-il)"], 0.3, 4, -3),
]

## Create the turbofan model

In [4]:
turbofan = Turbofan(
    sections = sections,
    vanes_count = 3,
    center_hole_diameter = 1,
    hub_diameter = 2    
)

In [5]:
turbofan_model = turbofan.build_turbofan()
turbofan_model

CadViewerWidget(anchor=None, cad_width=800, glass=False, height=600, pinning=False, theme='light', title=None,…